# Gradient decent

## Exact Gradient Computation

Given a function f, sometimes we can compute its exact gradient at any x if f's derivative is easy to compute. For example, let $f(x)=2x^2-3x+\ln x$, where $x>0$. Please compute the derivative of f and report its gradient at $x=2$.

Your answer: 

## Numerical Gradient Computation [5 pts]

Instead of computing the derivative of a function, we can also estimate the gradient numerically with various methods. These methods are essential, especially when a callable function is not exposed due to privacy reasons, or it is hard to differentiate analytically. 

To numerically compute the gradient, the simple way is to follow Newton's difference quotient: $f'(x)=\lim_{h\to 0}{f(x+h)-f(x)\over h}$. Another two-point formula is to compute the slope through the points $(x-h,f(x-h))$ and $(x+h,f(x+h))$. Let us reuse the example function $f(x)=2x^2-3x+\ln x$ and test the precision of these two approaches. Define the function in the next cell, and try to compute its gradient via both methods at $x=2$. Range h value in [0.1,0.01,0.001,0.0001] and report all gradients calculated. Which method is more accurate, and why does it work better?

In [78]:
import math

def f(x):
    # Your code here
    return 2*x**2-3*x+math.log(x)

In [79]:
# Compute gradient using the first method (Newton's difference quotient)
h_s = [.1,.01,.001,.0001]
f_prime = [(f(2+h)-f(2))/h for h in h_s]

In [80]:
# Compute gradient using the second method 
f2_prime = [(f(2+h)-f(2-h))/(2*h) for h in h_s]

In [81]:
f_prime

[5.687901641694317, 5.518754151103744, 5.50187504165045, 5.5001875004201395]

In [82]:
f2_prime

[5.500417292784909, 5.500004166729067, 5.500000041665842, 5.500000000417948]

#### Remark: You may find the gradient more accurate when using a smaller h value. However, this is not always the case. Due to the finite precision of the floating-point, rounding errors always exist and can dominate the computation when the h value is too small. Run the following two cells to observe such scenarios.

In [83]:
eps = 1e-15
print((f(2+eps)-f(2-eps))/2/eps)

5.551115123125783


In [84]:
eps = 1e-20
print((f(2+eps)-f(2-eps))/2/eps)

0.0


## Logistic regression

Logistic regression is a classification tool that models the probability of an event taking place by having the log odds for the event be a linear combination of one or more independent variables. Specifically, let $\vec{x}=<x_1,\dots ,x_m>$ be an m dimensional vector of independent variables (features), and $y$ be the corresponding binary dependent variable (label). The probability of having $y=1$ is modeled as $$P_y={1\over 1+e^{-(b_0+b_1\cdot x_1+\dots +b_m\cdot x_m)}}={1\over 1+e^{-(b_0+\vec{b}_{1:m}\cdot\vec{x})}}$$

Given a set of data points $<\vec{x}_k,y_k>$ with $k\in [1,n]$, how can we fit the model with these data, i.e., how to choose the best $\vec{b}=b_0,b_1\cdots,b_m$?

One way is to write out the likelihood $$\prod_{k:y_k=1}P_{y_k}\prod_{k:y_k=0}(1-P_{y_k})$$ and find $b_0,b_1\cdots,b_m$ that maximize its logarithm, $$l=\sum_{k:y_k=1}\ln(P_{y_k})+\sum_{k:y_k=0}\ln(1-P_{y_k})$$

In contrast to computing the closed form gradient of a Least-squares loss in a linear model (chapter 5 of MML book), doing the same for logistic regression is not possible. Gradient descent can be used to optimize such function $l$, and we will implement it step-by-step. First, write a function log_likelihood in the next cell that computes the log-likelihood given data points and $\vec{b}$. [5 pts]

In [85]:
import numpy as np
import sklearn

In [86]:
def p(X,b):
    #print("X shape: ",X.shape,"b shpae: ",b.shape)
    pp = X@b
    return 1/(1+math.exp(-1*pp))
    
def log_likelihood(X,y,b):
    """
    X: n*m numpy data array.
    y: one dimension numpy data array of length n
    b: one dimension numpy data array of length m+1
    
    Return the log likelihood.
    """
    n,m = X.shape
    X0 = np.ones((n,1))
    Xnew = np.hstack((X0,X))
    
    l = [math.log(p(Xnew[i],b)) if y[i]==1 else math.log(1-p(Xnew[i],b)) for i in range(len(y))]
    return sum(l)

### Test your log_likelihood function with a small example below.

In [87]:
X=np.array([[0.1],[0.5],[1.]])
y=np.array([0,0,1])
b=np.array([0.,1.])
# Your answer should be around -2.03
log_likelihood(X,y,b)

-2.031735331771901

In [88]:
b_new = b.copy()
b_new[0] = b_new[0]+.0001
print(b_new,b)

[1.e-04 1.e+00] [0. 1.]


Now that we have a function to maximize, the next step is to compute the gradient of the log-likelihood with respect to parameter $\vec{b}$. Use the method with Newton's difference quotient, and set $h=0.0001$. Implement the function compute_gradient in the next cell. [7 pts]

In [89]:
def compute_gradient(X,y,b):
# The inputs are the same as the ones of log_likelihood
    h = .0001
    res = []
    for i in range(len(b)):
        b_new = b.copy()
        
        b_new[i] = b_new[i]+h
        #print('b:',b," b_new: ",b_new)
        res.append((log_likelihood(X,y,b_new)-log_likelihood(X,y,b))/h)
    return res

In [90]:
# Test your function here, preserve the output
compute_gradient(X,y,b)

[-0.8785311466219525, -0.09479905564102609]

Once we know how to compute the gradients, we can optimize the objective, which is log-likelihood in our case, using gradient descent. It iteratively changes the parameters in a small "step" towards the gradient direction, i.e., the direction where the objective increases at the fastest pace. Formally, denote the calculated gradients as $\Delta (\vec{b})$, we can update our parameters via $\vec{b}=\vec{b}+\gamma \cdot \Delta (\vec{b})$, where $\gamma$ is the size of the "step". Repeat this process until the objective stop improving or a pre-set max number of iterations is reached. **Note in practice, the value of gradient changes over iterations and can be very large/small, so you should normalize the gradient vector every iteration, i.e., scale it to $\Delta (\vec{b})\over ||\Delta (\vec{b})||_2$, before using it to compute the new $\vec{b}$. Therefore, the update rule for parameters becomes $\vec{b}=\vec{b}+\gamma \cdot {\Delta (\vec{b})\over ||\Delta (\vec{b})||_2}$**.

Implement the gradient_descent function below. [7 pts]

In [103]:
def gradient_descent(X, y, initial_b, step_size, max_iteration):
    """
    X: n*m numpy data array.
    y: one dimension numpy data array of length n
    initial_b: one dimension numpy data array of length m+1
    step_size: scalar, the size of one step update
    max_iteration: scalar, the max number of iterations
    Return the updated coefficient vector b.
    """
    pre_ll = log_likelihood(X,y,initial_b)
    
    b = initial_b.copy()
    prev_b = b
    iter = 0
    for i in range(max_iteration):
        iter+=1
        grad = np.array(compute_gradient(X,y,b))
        grad = grad/np.sqrt(grad.T@grad)
        b = b + step_size * grad
        temp = log_likelihood(X,y,b)
        if(temp < pre_ll):
            iter-=1
            return prev_b, iter
        pre_ll = temp
        prev_b = b
        
    return b, iter

Test the function with the previous example again. Print for each sample from X, based on your model, the probability of having label=1.

In [104]:
optimized_b, iter  = gradient_descent(X, y, b, 0.1, 1000)
print(optimized_b,iter)
# compute and print the probability for each row in X below using optimized_b
n,m = X.shape
X0 = np.ones((n,1))
Xnew = np.hstack((X0,X))
[1/(1+math.exp(-1*(optimized_b@Xnew[i].T))) for i in range(len(Xnew))]

[-60.22500404  80.46041218] 1000


[2.182550875139692e-23, 2.07190378157202e-09, 0.9999999983711769]

Next, we apply the implemented logistic regression model to a real dataset. The dataset is a trimmed breast-cancer-Wisconsin dataset from UCI machine learning Repository. Only 100 data points are offered in the training set to make sure the computation can be finished swiftly, no matter how you implement the optimizer. The training dataset is loaded in the next cell, and the vector $\vec{b}$ is also randomly initialized. 

Fit three models with the training set using different step size ranging in [0.01,0.05,0.1] and set the max number of iterations as 10000. How do the final log-likelihood value and the number of iterations change with different step sizes? [7 pts]

In [105]:
f = open("breast-cancer-wisconsin.data","r")
X_train = []
y_train = []
for line in f:
    tmp = []
    for part in line.strip().split(",")[1:-1]:
        tmp.append(float(part))
    y_train.append((0 if line.strip().split(",")[-1]=="2" else 1))
    X_train.append(tmp)
X_train = np.array(X_train)
y_train = np.array(y_train)
random_b = np.random.uniform(0,1,size=(10))

In [97]:
import tqdm

In [106]:
# Fit three models with different step size, report the final log-likelihood, 
# number of iterations and the final coefficent vector b.
model_list = []
for step in [.01,.05,.1]:
    opt_b, itera = gradient_descent(X_train, y_train, random_b, step, 10000)
    print('opt_b; ',opt_b, 'iteration : ',itera)
    model_list.append(opt_b)
    l = log_likelihood(X_train,y_train,opt_b)
    print('likehood:',l,'\n')

opt_b;  [-1.49487808e+01  9.48546833e-01 -1.41683547e+00  9.29835310e-01
  9.95595892e-01  4.67700382e-01 -4.79715208e-04  8.66934216e-01
  1.05312394e+00  1.44183208e+00] iteration :  6551
likehood: -7.4853626398190425 

opt_b;  [-2.90973277  0.19750722  0.22915517  0.14809109  0.14747135  0.02708475
  0.17568609 -0.43456428  0.41038245  0.09423698] iteration :  264
likehood: -20.180316115259846 

opt_b;  [ 0.14352998 -0.05652715  0.16623522  0.16805258  0.07503815 -0.06004591
  0.1457018  -0.69657107  0.50223143 -0.06030333] iteration :  33
likehood: -45.47460238439105 



Finally, load the test dataset, and predict for each sample in the test set what labels it should have using the model obtained. Compare your results with the ground truth labels, and report the accuracy rate. [4 pts]

In [99]:
f = open("test_data.txt","r")
X_test = []
y_test = []
for line in f:
    tmp = []
    for part in line.strip().split(",")[1:-1]:
        tmp.append(float(part))
    y_test.append((0 if line.strip().split(",")[-1]=="2" else 1))
    X_test.append(tmp)

In [121]:
# Predict based on your models and report the accuracy
for b in model_list:
    n,m = np.array(X_test).shape
    X0 = np.ones((n,1))
    Xnew = np.hstack((X0,X_test))
    prob = [1/(1+math.exp(-1*(b@Xnew[i].T))) for i in range(len(Xnew))]
    pred = np.round(prob)
    accuracy = np.mean(np.abs(np.subtract(np.array(pred),np.array(y_test))))
    print(1-accuracy)

0.9
0.9
0.7
